# New PxS

New Python x Smile runner, that uses a completely external standalone cpp script for maximum robustness. If it is stupid and it works, it is not stupid.

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import json


from os.path import dirname

In [3]:
# Custom imports
root_dir = dirname(os.getcwd())
root_dir

'/cw/dtailocal/repos/PxS'

In [10]:
sys.path.append(src_dir)

import exp
from exp.utils.run import run_script, generate_command, run_process,generate_monitor

ModuleNotFoundError: No module named 'exp'

## Definition

In [3]:
class PxS(object):
    
    default_log_fn = "PxS_log"
    default_to = 360
    default_cwd = os.getcwd()
    default_cfg_fn = "config.json"
    default_res_fn = "out.csv"
    default_model_fname = "model.xdsl"
    
    fit_fn = "./resc/fit"
    predict_fn = "./resc/predict"
    

    def __init__(self):
        return
    
    def gen_fit_cfg(self, train_fname, model_fname=None):
        
        if model_fname is None:
            model_fname = self.default_model_fname
        
        cfg = {"train_fname": train_fname,
               "model_fname": model_fname}
        
        return cfg
    
    def gen_predict_cfg(self, train_fname, targ_idx, miss_idx, out_fname=None, model_fname=None):
        
        if out_fname is None:
            out_fname=self.default_res_fn
        if model_fname is None:
            model_fname = self.default_model_fname
        
        cfg = {"test_fname":  train_fname,
               "out_fname":   out_fname,
               "model_fname": model_fname,
               "miss_idx":    miss_idx,
               "targ_idx":    targ_idx}
        
        return cfg
      
    def save_config(self, cfg, cfg_fname=None):
        if cfg_fname is None:
            cfg_fname = self.default_cfg_fn
        
        with open(cfg_fname, 'w') as f:
            json.dump(cfg, f, indent=4)
        return
    
    def drop_log(self, log_fname, cwd):
        
        success_log_fnames = [os.path.join(cwd, f) for f in os.listdir(cwd)
                             if log_fname in f
                             if "success" in f]
        
        for f in success_log_fnames:
            os.remove(f)      
        
        return
     
    def fit(self, train_fname, model_fname=None, cfg_fname=None, log_fname=None, timeout=None, cwd=None):
        
        # Parse arguments
        if log_fname is None:
            log_fname = self.default_log_fn+"_fit"
        if timeout is None:
            timeout = self.default_to
        if cwd is None:
            cwd = self.default_cwd
        if cfg_fname is None:
            cfg_fname = self.default_cfg_fn
            
        # Config
        cfg = self.gen_fit_cfg(train_fname, model_fname=model_fname)
        self.save_config(cfg, cfg_fname=cfg_fname)
        
        # Run - Prelims
        mon = generate_monitor(log_fname, timeout)
        cmd = generate_command(self.fit_fn, 
                               cfg_fname, 
                               script_prefix="",
                               config_prefix="-c")
        
        # Run 
        p = run_process(cmd, monitors=mon, cwd=cwd)
        
        if p == 0:
            os.remove(cfg_fname)
            self.drop_log(log_fname, cwd)
            
        return p
    
    def predict(self,
                test_fname,
                targ_idx,
                miss_idx,
                out_fname=None,
                model_fname=None,
                cfg_fname=None,
                log_fname=None,
                timeout=None,
                cwd=None):
        
        # Parse arguments
        if log_fname is None:
            log_fname = self.default_log_fn+"_predict"
        if timeout is None:
            timeout = self.default_to
        if cwd is None:
            cwd = self.default_cwd
        if cfg_fname is None:
            cfg_fname = self.default_cfg_fn
            
        
        # Config
        cfg = self.gen_predict_cfg(test_fname, targ_idx, miss_idx, out_fname=out_fname, model_fname=model_fname)
        self.save_config(cfg, cfg_fname=cfg_fname)
            
        # Run-Prelims
        mon = generate_monitor(log_fname, timeout)
        cmd = generate_command(self.predict_fn, 
                               cfg_fname, 
                               script_prefix="",
                               config_prefix="-c")
        
        # Run 
        p = run_process(cmd, monitors=mon, cwd=cwd)
        
        if p == 0:
            os.remove(cfg_fname)
            self.drop_log(log_fname, cwd)
            
            result = pd.read_csv(cfg["out_fname"], header=None)
            #os.remove(cfg["out_fname"])
        
            return result.values
        else:
            return p
    

## Factory Floor

Here I try the crap I write

In [4]:
pxs = PxS()

In [22]:
pxs.fit(train_fname='resc/train.csv')

0

In [64]:
%time
res = pxs.predict(test_fname="resc/test.csv", targ_idx=[10,11,12], miss_idx=[4])
res

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 9.78 µs


array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0]])

array([-1])

In [7]:
log_fn = os.path.join(os.getcwd(), "prod/log.txt")
log_fn = "prod/log.txt"

In [8]:
mon = generate_monitor(log_fn, 360)

In [9]:
cmd = generate_command(pxs.fit_cpp, "resc/config.json", script_prefix="", config_prefix="-c")
cmd

AttributeError: 'PxS' object has no attribute 'fit_cpp'

In [ ]:
run_process(cmd, monitors=mon, cwd=os.getcwd())